<a href="https://colab.research.google.com/github/pndang/Project_WHEL/blob/main/random_forest_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Author: Phu Dang
# Date: 03.02.2023
# Mentor: Professor Niema Moshiri, Niema Lab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [45]:
# Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from pandas.plotting import scatter_matrix
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

In [6]:
cd /content/drive/MyDrive/NIEMA_Lab/WHEL_Data/Preprocessed-Data

/content/drive/MyDrive/NIEMA_Lab/WHEL_Data/Preprocessed-Data


In [7]:
# Loading dataset

df = pd.read_csv('ovrt.csv')
df.head()

,Unnamed: 0,ID,QOL_ovt,energy_ovt,genhlth_ovt,optimism_ovt,pain_ovt,socfctn_ovt,lutein_ovt,cryptox_ovt,...,water_ovt,anim. protein_ovt,veg prot_ovt,vit k_ovt,trans fat acids_ovt,beta-carot_ovt,alpha-carot_ovt,beta-cryptox_ovt,lut + zeax_ovt,lycopene_ovt_y
0,0,1245,-32.312500,-20.0,-40.0,-2.0,-12.5,-50.0,-0.1697,-0.0465,...,193.992493,-1.905000,8.29750,91.972500,-0.3525,1399.397461,24.567507,54.597500,1790.035065,250.457520
1,1,1256,-36.687500,-5.0,10.0,-3.0,-50.0,-37.5,0.0980,-0.0966,...,722.049988,-5.290000,-6.13250,7.219999,-0.4350,-311.827454,248.392479,15.737500,-347.647522,-1278.749996
2,2,1265,-6.312500,-5.0,-10.0,2.0,-12.5,0.0,0.0349,-0.1795,...,499.857483,-38.457500,1.31500,34.244999,-1.4150,18381.732422,8353.942706,61.490000,2007.082550,26804.000000
3,3,1266,0.437500,-10.0,0.0,2.0,25.0,-12.5,-0.1125,-0.1744,...,299.345032,35.095001,-5.08750,26.660002,1.3200,-5628.977448,-1542.992467,312.769993,2945.134918,4928.824951
4,4,1276,30.646484,30.0,5.0,-1.0,25.0,37.5,0.0766,0.0213,...,51.471191,4.437499,-1.35375,-67.597503,0.7600,49091.856445,28144.247949,284.864999,92.692444,-1542.129974


In [8]:
df.shape

(655, 69)

In [9]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [10]:
df

,ID,QOL_ovt,energy_ovt,genhlth_ovt,optimism_ovt,pain_ovt,socfctn_ovt,lutein_ovt,cryptox_ovt,lycopene_ovt_x,...,water_ovt,anim. protein_ovt,veg prot_ovt,vit k_ovt,trans fat acids_ovt,beta-carot_ovt,alpha-carot_ovt,beta-cryptox_ovt,lut + zeax_ovt,lycopene_ovt_y
0,1245,-32.312500,-20.0,-40.0,-2.0,-12.5,-50.0,-0.1697,-0.0465,0.3556,...,193.992493,-1.905000,8.297500,91.972500,-0.3525,1399.397461,24.567507,54.597500,1790.035065,250.457520
1,1256,-36.687500,-5.0,10.0,-3.0,-50.0,-37.5,0.0980,-0.0966,0.0129,...,722.049988,-5.290000,-6.132500,7.219999,-0.4350,-311.827454,248.392479,15.737500,-347.647522,-1278.749996
2,1265,-6.312500,-5.0,-10.0,2.0,-12.5,0.0,0.0349,-0.1795,-0.4249,...,499.857483,-38.457500,1.315000,34.244999,-1.4150,18381.732422,8353.942706,61.490000,2007.082550,26804.000000
3,1266,0.437500,-10.0,0.0,2.0,25.0,-12.5,-0.1125,-0.1744,-0.4923,...,299.345032,35.095001,-5.087500,26.660002,1.3200,-5628.977448,-1542.992467,312.769993,2945.134918,4928.824951
4,1276,30.646484,30.0,5.0,-1.0,25.0,37.5,0.0766,0.0213,-0.0547,...,51.471191,4.437499,-1.353750,-67.597503,0.7600,49091.856445,28144.247949,284.864999,92.692444,-1542.129974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,13237,-2.875000,5.0,-15.0,1.0,0.0,0.0,-0.4909,0.0686,0.7054,...,778.665039,-15.582501,-2.377500,28.867503,-4.2725,953.305054,964.039978,76.027504,299.722534,17455.123047
651,13241,1.437500,5.0,5.0,-1.0,12.5,0.0,0.0065,0.0827,0.4094,...,-394.652527,8.437500,-2.780000,-3.692498,0.9900,-1166.187500,458.790023,-1468.800001,-799.239998,-7753.884999
652,13242,2.187500,5.0,5.0,2.0,-12.5,0.0,0.0932,-0.1789,0.2133,...,-738.780029,-34.210000,5.542500,-16.477497,0.3250,-2668.910055,-193.804963,-299.970001,-1525.382530,1057.294678
653,13243,9.937500,15.0,10.0,1.0,12.5,0.0,-0.0045,0.1245,-0.1687,...,392.342163,-40.800000,3.721312,122.134993,-3.4975,5834.960815,1415.401793,249.318362,3406.976135,20269.307617


In [17]:
df.groupby(by='recurrence').count().get('ID')

recurrence
0    629
1     26
Name: ID, dtype: int64

In [18]:
!ls

bcep_tf.csv  ovrt.csv  ovrt_phu.csv


In [20]:
df_phu = pd.read_csv('ovrt_phu.csv')
df_phu.head()

,Unnamed: 0,ID,QOL_ovt,energy_ovt,genhlth_ovt,optimism_ovt,pain_ovt,socfctn_ovt,brcastatus,brcastatus_binary,lutein_ovt,cryptox_ovt,lycopene_ovt,alphacar_ovt,betacar_ovt
0,0,1018,-15.750000,5.0,0.0,0.0,-37.5,-12.5,0,0,-0.0347,0.0303,-0.0820,0.0918,0.8330
1,1,1019,1.855469,15.0,-15.0,0.0,-12.5,0.0,0,0,0.4003,0.0880,-0.0739,0.1937,0.3823
2,2,1022,-2.312500,0.0,-10.0,-2.0,-12.5,0.0,0,0,0.0056,-0.0503,-0.2200,0.0528,0.3148
3,3,1026,-7.769531,5.0,-5.0,2.0,0.0,-37.5,0,0,-0.1215,-0.3689,-0.3149,0.0722,0.3063
4,4,1027,9.625000,20.0,0.0,-1.0,12.5,37.5,0,0,0.1232,0.1394,0.4131,0.0773,0.9563


In [22]:
df_phu.groupby(by='brcastatus_binary').count().get('ID')

brcastatus_binary
0    1741
1     103
Name: ID, dtype: int64

In [28]:
np.arange(0, 68)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67])

In [27]:
df.columns

Index(['ID', 'QOL_ovt', 'energy_ovt', 'genhlth_ovt', 'optimism_ovt',
       'pain_ovt', 'socfctn_ovt', 'lutein_ovt', 'cryptox_ovt',
       'lycopene_ovt_x', 'alphacar_ovt', 'betacar_ovt', 'recurrence',
       'heightcm_ovt', 'weightkg_ovt', 'bmi_ovt', 'waistcm_ovt', 'hipcm_ovt',
       'pulse30_ovt', 'systolic_ovt', 'diastolic_ovt', 'anti-est_use_ovt',
       'anti-estro_type_ovt', '% protein_ovt', '% fat_ovt', '% carb_ovt',
       '% alcohol_ovt', 'caffeine_ovt', 'calcium_ovt', 'iron_ovt',
       'magnesium_ovt', 'phosphorus_ovt', 'potassium_ovt', 'sodium_ovt',
       'zinc_ovt', 'copper_ovt', 'selenium_ovt', 'alpha-toc_ovt',
       'gamma-toc_ovt', 'vit d_ovt', 'vitamin c_ovt', 'thiamin_ovt',
       'riboflavin_ovt', 'niacin_ovt', 'folacin_ovt', 'panto. acid_ovt',
       'vit b6_ovt', 'vit b12_ovt', 'cholesterol_ovt', 'diet fiber_ovt',
       'soluble fiber_ovt', 'insol. fiber_ovt', 'starch_ovt', 'sucrose_ovt',
       'galactose_ovt', 'fructose_ovt', 'lactose_ovt', 'maltose_ovt',
   

In [35]:
X = df.drop(['ID', 'recurrence'], axis=1)

# Converting data into numpy arrays
X = X.to_numpy()
y = df['recurrence']

In [37]:
len(X)

655

In [38]:
len(y)

655

In [48]:
# Spltting dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.5)

In [49]:
# Initiating model
clf = RandomForestClassifier(n_estimators = 100)

# Training model
clf.fit(X_train, y_train)

RandomForestClassifier()

In [50]:
# Make predictions on the test set

y_preds = clf.predict(X_test)

In [51]:
# Get accuracy

acc = metrics.accuracy_score(y_test, y_preds)

print(f'Model accuracy: {acc}')

Model accuracy: 0.9542682926829268
